In [1]:
from pyproj import Proj, transform
def wgs84_to_svy21(latitude, longitude):
    # Convert from WGS84 to SVY21
    easting, northing = transform(wgs84_proj, svy21_proj, longitude, latitude)
    return northing, easting
    
# Define the projection for SVY21
svy21_proj = Proj(init='epsg:3414')  # EPSG code for SVY21

# Define the projection for WGS84
wgs84_proj = Proj(init='epsg:4326')  # EPSG code for WGS84

def svy21_to_wgs84(northing, easting):
    # Convert from SVY21 to WGS84
    lon, lat = transform(svy21_proj, wgs84_proj, easting, northing)
    return lat, lon

# Example usage
# northing = 31490.4942
# easting = 30800.0120
# latitude, longitude = svy21_to_wgs84(northing, easting)
# print(f"Latitude: {latitude}, Longitude: {longitude}")


/home/alexy/miniconda3/envs/airflow/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/alexy/miniconda3/envs/airflow/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [2]:
# Download the bus_stop.xml file
import requests
url = 'https://www.lta.gov.sg/map/busService/bus_stops.xml'
response = requests.get(url)
with open('bus_stops.xml', 'wb') as f:
    f.write(response.content)
    print('File downloaded successfully')

File downloaded successfully


In [9]:
# Extract Data from bus_stops.xml

import pandas as pd
import xml.etree.ElementTree as ET

# Load and parse the XML file
tree = ET.parse('bus_stops.xml')
root = tree.getroot()

# Prepare a list to hold all bus stop data
bus_stops = []

# Iterate through each bus stop in the XML
for bus_stop in root.findall('.//busstop'):
    # print(bus_stop)
    # Extract elements from each BusStop
    bus_stop_id = bus_stop.find('details').text if bus_stop.find('details') is not None else None
    bus_stop_cor = bus_stop.find('coordinates') if bus_stop.find('coordinates') is not None else None
    # print(type(bus_stop。or))
    # print(bus_stop_cor)
    latitude = bus_stop_cor.find('lat').text if bus_stop_cor.find('lat') is not None else None
    longitude = bus_stop_cor.find('long').text if bus_stop_cor.find('long') is not None else None
    
    # x, y = wgs84_to_svy21(float(latitude), float(longitude))
    # Append to list
    bus_stops.append({
        'BusStopID': bus_stop_id,
        # 'BusStopCode': bus_stop_code,
        # 'Description': description,
        'Latitude': latitude,
        'Longitude': longitude,
    })

# Convert list to DataFrame
df = pd.DataFrame(bus_stops)

# remove same detail bus stop
df = df.drop_duplicates(subset=['BusStopID'])

# reorder index
df = df.reset_index(drop=True)

# Print DataFrame to see the result
print(df)

# Save DataFrame to CSV
df.to_csv('bus_stops.csv', index=False)



                      BusStopID          Latitude           Longitude
0       Outside Estella Gardens  1.35986800000000  103.96568100000000
1                 Raffles Hotel  1.29452091230981  103.85402155505173
2        The Float @ Marina Bay  1.28951669800000  103.85891720000000
3                 The Esplanade  1.28987200800000  103.85649660000000
4     Promenade Stn/Pan Pacific  1.29153323500000  103.85926730000000
...                         ...               ...                 ...
4721                      Blk 5  1.38819494717642  103.98723382929484
4722      Bef Changi Beach CP 3  1.39026180023806  103.99295664134459
4723          Changi Beach CP 2  1.39112750002185  103.99102138892110
4724     Bef S'pore Aviation Ac  1.38775412603923  103.98850309975099
4725         S'pore Aviation Ac  1.38841416669580  103.98971638885320

[4726 rows x 3 columns]
